<a href="https://colab.research.google.com/github/utechs/Mo/blob/main/lda_on_instagram_post_captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install some required libraries

In [5]:
!pip3 install pyLDAvis ##chart library
!pip3 install contractions ##This package is capable of resolving contractions
!pip3 install clean-text[gpl]
!pip3 install google-cloud-vision ## image processing library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 6.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive') ## giving colab access to your drive

# Use Google vision api to get images lables
Please note this paid service don't use it frequently

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import os

def detect_labels_uri(uri):
    """Detects labels in the file located in Google Cloud Storage or on the
   Web."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.label_detection(image=image)
    labels = response.label_annotations
    return labels

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "/content/drive/My Drive/key.json" # you need to generate your own key to use google vision api
df = pd.read_excel("/content/drive/My Drive/influencers-data.xlsx") # file contain images urls which collected locally on my pc using some custom script
# Start of Function 
def extract_labels_from_gvision(url):
    my_labels = detect_labels_uri(url)
    my_label_list = list()
    for label in my_labels:
        my_label_list.append(label.description)
    return my_label_list
# End of Function
df['labels'] = df['display_url'].apply(extract_labels_from_gvision) # excute the extract_labels_from_gvision function on all image url
df.to_excel("/content/drive/My Drive/influencers-data-with-images-lables.xlsx",index=False) # append lables to pur data and save it to new file

# NLP FOR IMAGES LABLES

***This section still under development***

*   Read data data from excel sheet
*   apply some nlp function on the images lables
*   caculate  engagement score using likes and comments score



In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
from pprint import pprint
import spacy
from nltk.tokenize import word_tokenize

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

df = pd.read_excel("/content/drive/My Drive/with-images-lables.xlsx") # read data with lables from excel file

df['labels'] = df['labels'].apply(lambda x: literal_eval(x))
df["labels"] = df["labels"].astype(str)
df["labels"] = df["labels"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["labels_strings"] = df['labels'].apply(' '.join)


df["likes_normalized"] = df["likes"]/df["likes"].max() 
df["comments_normalized"] = df["comments"]/df["comments"].max()

# Create engagement score we could change this later
df["engagement_score"] = 0.4*df["likes_normalized"] + 0.6*df["comments_normalized"]
engagement_median = df["engagement_score"].median()
df["engagement"] = df["engagement_score"].apply(lambda x: 1 if x > engagement_median else 0)
df.head() # used to show results in console
df.to_excel("/content/drive/My Drive/with-images-lables-cleaned.xlsx",index=False)


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


FileNotFoundError: ignored

In [ ]:
import nltk
nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint

from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
import urllib.request 
import statsmodels.api as sm
import math
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

df = pd.read_excel("/content/drive/My Drive/suzannaak1-res1.xlsx")
# Keeping all labels intact and using tf-idf score
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['labels_strings'].values.astype('U').tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagement'] = df['engagement']


X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagement'], test_size=0.20, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train,y_train )
print("Accuracy with Image Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))

# Keeping all captions intact and using tf-idf score
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['caption'].values.astype('U').tolist())

q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = df['engagement']

X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.20, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with Caption Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))

df['caption+labels'] = df['caption'] + " " + df['labels_strings']

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['caption+labels'].values.astype('U').tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = df['engagement']

X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.33, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with both Captions and Image Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
import numpy as np
import pandas as pd
from pprint import pprint

from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import DataFrame, Series
import urllib.request 
import statsmodels.api as sm
import math
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df = pd.read_excel("/content/drive/My Drive/suzannaak1-res1.xlsx")
df["labels"] = df["labels"].astype(str)
df["label_tokens"] = df["labels"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["label_tokens"] = df["label_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stop_words])
df.head()

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod
def get_corpus(df):
    """
    Get Bigram Model, Corpus, id2word mapping
    """
    bigram = bigrams(df.label_tokens)
    bigram = [bigram[review] for review in df.label_tokens]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram
train_corpus, train_id2word, bigram_train = get_corpus(df)
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import warnings
import  multiprocessing
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    lda_train = gensim.models.ldamulticore.LdaMulticore(
                            corpus=train_corpus,
                            num_topics=4,
                            id2word=train_id2word,
                            chunksize=100,
                            workers=7, # Num. Processing Cores - 1
                            passes=80,
                            eval_every = 1,
                            per_word_topics=True)
    lda_train.save('lda_train.model')
    coherence_model_lda = CoherenceModel(model=lda_train, texts=bigram_train, dictionary=train_id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print("The coherence of the LDA model is",coherence_lda)
    vis = pyLDAvis.gensim_models.prepare(topic_model=lda_train, corpus=train_corpus, dictionary=train_id2word)
    pyLDAvis.save_html(vis, '/content/drive/My Drive/5S2lda.html')
    df_lda = pd.DataFrame(lda_train.show_topics(), columns=['Topic','Word Weights'])
    df_lda.head()
    df_lda.to_excel('/content/drive/My Drive/5S2Word_Weights.xlsx')
    lda_train.show_topics()

# NLP FROM POSTS CAPTIONS


*   Read data data from excel sheet
*   Caculate engagement score using likes and comments


---

*   Apply some nlp function on the images lables




In [3]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

df = pd.read_excel("/content/drive/My Drive/influencers-data.xlsx") ## all influencers data
df["likes_normalized"] = df["likes"]/df["likes"].max() 
df["comments_normalized"] = df["comments"]/df["comments"].max()
## Create engagement score
df["engagement_score"] = 0.4*df["likes_normalized"] + 0.6*df["comments_normalized"]
engagement_median = df["engagement_score"].median()
df["engagement"] = df["engagement_score"].apply(lambda x: 1 if x > engagement_median else 0)
df.to_excel("/content/drive/My Drive/influencers-data-engagement-score.xlsx",index=False)

In [6]:
import contractions
from cleantext import clean
import pandas as pd
import numpy as np
import spacy


import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
df = pd.read_excel("/content/drive/My Drive/influencers-data-engagement-score.xlsx")
# start hashtags functions
def extract_hashtags(text):
  textList = text.split()
  hashtags = [item for item in textList if item[0] == "#"]
  return hashtags

def remove_hashtags(text):
  textList = text.split()
  text =  [item for item in textList if item[0] != "#"]
  return (' ').join(text)

# end hashtags functions

#df['caption'] = df['caption'].apply(lambda x: literal_eval(x))
df["caption"] = df["caption"].astype(str)
df["hashtags"] = df["caption"].apply(lambda each_post: extract_hashtags(each_post))
df["caption_without_hash"] = df["caption"].apply(lambda each_post: remove_hashtags(each_post))
df["caption_without_hash"] = df["caption_without_hash"].apply(lambda each_post: contractions.fix(each_post))
df["caption_without_hash"] = df["caption_without_hash"].apply(lambda each_post: clean(each_post, no_emoji=True))

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
doc = df["caption_without_hash"].apply(lambda each_post: nlp(each_post))
df["caption_lemma"] = doc.apply(lambda each_post: (" ".join([token.lemma_ for token in each_post])))

df.to_excel("/content/drive/My Drive/influencers-data-cleaned.xlsx",index=False) 


In [11]:
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
import re
from pprint import pprint
# spacy for STOP WORDS
sp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
spacy_stopwords = sp.Defaults.stop_words
words = set(words.words())
spacy_stopwords |= {"jest","sie", "nan"} # Add to stop words list
df = pd.read_excel("/content/drive/My Drive/influencers-data-cleaned.xlsx") # Read file from drive

df["caption_lemma"] = df["caption_lemma"].astype(str)
df["caption_lemma_clean"] = df["caption_lemma"].apply(lambda each_post: re.sub(r"[^a-zA-Z0-9 ]", "", each_post))
df["caption_tokens"] = df["caption_lemma_clean"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["caption_tokens"] = df["caption_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in spacy_stopwords])
df["caption_tokens"] = df["caption_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x in words or not x.isalpha()])
df["caption_tokens"] = df["caption_tokens"].apply(lambda list_of_words: [x for x in list_of_words if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())])
df["caption_tokens"] = df["caption_tokens"].apply(lambda list_of_words: [x for x in list_of_words if len(x) > 2 ])

df.to_excel('/content/drive/My Drive/influencers-data-with-tokens.xlsx') # Save file to drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import DataFrame, Series
import urllib.request 
import statsmodels.api as sm
import math

df = pd.read_excel("/content/drive/My Drive/influencers-data-with-tokens.xlsx")

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod
def get_corpus(df):
    """
    Get Bigram Model, Corpus, id2word mapping
    """
    bigram = bigrams(df.caption_tokens)
    bigram = [bigram[review] for review in df.caption_tokens]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram
train_corpus, train_id2word, bigram_train = get_corpus(df)
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import warnings
import  multiprocessing
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                            corpus=train_corpus,
                            num_topics=4,
                            id2word=train_id2word,
                            chunksize=100,
                            workers=7, # Num. Processing Cores - 1
                            passes=80,
                            eval_every = 1,
                            per_word_topics=True)
    lda_train.save('lda_train.model')
    coherence_model_lda = CoherenceModel(model=lda_train, texts=bigram_train, dictionary=train_id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print("The coherence of the LDA model is",coherence_lda)
    vis = pyLDAvis.gensim_models.prepare(topic_model=lda_train, corpus=train_corpus, dictionary=train_id2word)
    pyLDAvis.save_html(vis, '/content/drive/My Drive/chart-lda.html')
    df_lda = pd.DataFrame(lda_train.show_topics(), columns=['Topic','Word Weights'])
    df_lda.head()
    df_lda.to_excel('/content/drive/My Drive/Word_Weights.xlsx')
    lda_train.show_topics()

The coherence of the LDA model is 0.642476230043491


In [ ]:
train_vecs = []
for i in range(len(df.label_tokens)):
  top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0)
  topic_vec = [top_topics[i][1] for i in range(4)]
  train_vecs.append(topic_vec)
train_vec_df=pd.DataFrame(train_vecs)
train_vec_df.columns=['topic0','topic1','topic2','topic3']
train_vec_df.iloc[31]
df_nat_final=pd.concat([df.reset_index(drop=True), train_vec_df.reset_index(drop=True)], axis=1)
df_nat_final[:2]
df_nat_final.to_excel("/content/drive/My Drive/STopic_Weights.xlsx")
q1=np.percentile(df_nat_final.engagement_score, 25) 
q2=np.percentile(df_nat_final.engagement_score, 50)  
q3=np.percentile(df_nat_final.engagement_score, 75)

print (q1,q2,q3)
top_quartile=df_nat_final[df_nat_final['engagement_score']>q3]
top_quartile[:3]
average_topic_weights_top = top_quartile[["topic0",'topic1',"topic2",'topic3']].mean(axis=0)
average_topic_weights_top
bottom_quartile=df_nat_final[df_nat_final['engagement_score']<q1]
bottom_quartile[:3]
average_topic_weights_bot = bottom_quartile[["topic0",'topic1',"topic2",'topic3']].mean(axis=0)
average_topic_weights_bot
quartile_topics = pd.concat([average_topic_weights_top,average_topic_weights_bot],axis=1)
quartile_topics.columns = ['Top Quartile','Bottom Quartile']
quartile_topics

0.037846623244339364 0.07428245694367869 0.11016741269589107


,Top Quartile,Bottom Quartile
topic0,0.306054,0.279198
topic1,0.196484,0.217985
topic2,0.162380,0.312911
topic3,0.335082,0.189907
